In [3]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import feedparser
import os
import numpy as np
import time
import cchardet
import lxml

In [5]:
# Getting the current work directory (cwd)
thisdir = os.getcwd()
snapshots = []

# r=root, d=directories, f = files
for r, d, f in os.walk(thisdir+"/bbc_brazil"):
    for file in f:
        if file.endswith(".snapshot"):
            snapshots.append(os.path.join(r, file))

In [8]:
snapshots

['/Users/dat/Master-thesis/bbc_brazil/feeds.bbci.co.uk/portuguese/rss.xml/20170707091807.snapshot',
 '/Users/dat/Master-thesis/bbc_brazil/feeds.bbci.co.uk/portuguese/rss.xml/20181223115422.snapshot',
 '/Users/dat/Master-thesis/bbc_brazil/feeds.bbci.co.uk/portuguese/rss.xml/20190607124352.snapshot',
 '/Users/dat/Master-thesis/bbc_brazil/feeds.bbci.co.uk/portuguese/rss.xml/20190910090354.snapshot',
 '/Users/dat/Master-thesis/bbc_brazil/feeds.bbci.co.uk/portuguese/rss.xml/20160921203541.snapshot',
 '/Users/dat/Master-thesis/bbc_brazil/feeds.bbci.co.uk/portuguese/rss.xml/20160718191346.snapshot',
 '/Users/dat/Master-thesis/bbc_brazil/feeds.bbci.co.uk/portuguese/rss.xml/20160819182524.snapshot',
 '/Users/dat/Master-thesis/bbc_brazil/feeds.bbci.co.uk/portuguese/rss.xml/20180317231641.snapshot',
 '/Users/dat/Master-thesis/bbc_brazil/feeds.bbci.co.uk/portuguese/rss.xml/20200308075257.snapshot',
 '/Users/dat/Master-thesis/bbc_brazil/feeds.bbci.co.uk/portuguese/rss.xml/20190202193507.snapshot',


In [9]:
data = pd.DataFrame([])
for snap in snapshots:
    parser = feedparser.parse(snap)
    for post in parser.entries:
        try:
            title = post.title
            link = post.link
            date = post.published
            data = data.append(pd.DataFrame({'title': title, 'url': link,'date' : date}, index=[0]), ignore_index=True)
        except:
            continue
    

In [10]:
data.shape

(4952, 3)

In [18]:
data = data.drop_duplicates(subset='url')
data = data.dropna(subset=["url"])

In [19]:
data.shape

(4125, 3)

In [20]:
def categorize(x):
    if 'geral' in x:
        return "geral"
    elif 'internacional' in x:
        return "internacional"
    elif 'brasil' in x:
        return "brasil"
    else:
        return np.nan


In [21]:
data['category'] = data["url"].apply (lambda x: categorize(x))

In [1]:
def extract_userneed(x):
    try:
        page=requests.get(x)
        soup=BeautifulSoup(page.text,'lxml')
        x = [i for i in soup.find("script",text = re.compile(r'(?<="campaignName".").+?(?=\")'))][0]
        return re.search(r'(?<="campaignName".").+?(?=\")',x)[0]
    except:
        return np.nan

In [51]:
def get_text(url):
    try:
        requests_session = requests.Session()
        article = requests_session.get(url)
        soup = BeautifulSoup(article.content, 'lxml')
        body = soup.find(role='main')
        text = [p.text for p in body.find_all('p',dir="ltr")]
        return ''.join(text)
    except:
        return "error"

In [52]:
%%time
data["User Need"] = data["url"].apply(lambda x : extract_userneed(x))

CPU times: user 5min 47s, sys: 12.6 s, total: 6min
Wall time: 43min


In [53]:
data["User Need"] = data["User Need"].apply(lambda s: re.sub(r"(WS - )", "", s) if pd.notnull(s) else s)


In [54]:
%%time
data["text"] = data["url"].apply(lambda x : get_text(x))

CPU times: user 4min 38s, sys: 8.83 s, total: 4min 47s
Wall time: 13min 58s


In [56]:
data.to_csv("checkpoint.csv",index=False)

In [57]:
brazil1 = pd.read_csv("brazil_userneeds.csv")

In [61]:
brazil_final = brazil1.append(data,ignore_index=True)

In [62]:
brazil_final.shape

(7642, 6)

In [63]:
brazil_final = brazil_final.dropna(subset=["url","text"])
brazil_final = brazil_final.drop_duplicates(subset='url')


In [64]:
brazil_final.shape

(7638, 6)

In [66]:
brazil_final.to_csv("brazil_final.csv",index=False)